In [62]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import  OpenAI
from langchain.vectorstores import Pinecone
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
import os
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from dotenv import load_dotenv
load_dotenv()

True

In [60]:
folder_path="/Users/HP/genaivectordb/pdf/"
loader = PyPDFDirectoryLoader(folder_path)
documents = loader.load()




True

In [34]:
documents[0].metadata.get("source").split("\\")[5]

'NIPS.pdf'

In [42]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)

In [43]:
text_chunk= text_splitter.split_documents(documents)

In [45]:
text_chunk[1].page_content

'convolutional neural networks that include an encoder and a decoder. The best\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architecture, the Transformer,\nbased solely on attention mechanisms, dispensing with recurrence and convolutions\nentirely. Experiments on two machine translation tasks show these models to\nbe superior in quality while being more parallelizable and requiring signiﬁcantly'

In [54]:
print(text_chunk[0].page_content)

Attention Is All You Need
Ashish Vaswani∗
Google Brain
avaswani@google.comNoam Shazeer∗
Google Brain
noam@google.comNiki Parmar∗
Google Research
nikip@google.comJakob Uszkoreit∗
Google Research
usz@google.com
Llion Jones∗
Google Research
llion@google.comAidan N. Gomez∗†
University of Toronto
aidan@cs.toronto.eduŁukasz Kaiser∗
Google Brain
lukaszkaiser@google.com
Illia Polosukhin∗‡
illia.polosukhin@gmail.com
Abstract
The dominant sequence transduction models are based on complex recurrent or


In [51]:
len(text_chunk)

76

In [55]:
os.environ["OPENAP_API_KEY"]=""

In [64]:
embedding=OpenAIEmbeddings()

In [66]:
a= embedding.embed_query("How are you")

In [67]:
a

[-0.016044848920016213,
 -0.0028205322907030135,
 -0.004410748248726431,
 -0.036478250341766265,
 -0.02373115717395613,
 0.016387309295958042,
 -0.02866510525648591,
 -0.020420716807293363,
 -0.023249177214862467,
 -0.005555449953081893,
 0.025253197287396084,
 0.005615697215137939,
 -0.010311827868719032,
 0.0039573068095150295,
 -0.010121572621708375,
 -0.015512135159709024,
 0.043784048101684875,
 -0.009138587644481309,
 0.007756067579854512,
 -0.015042838883749405,
 0.002249766782501707,
 0.009798139167451583,
 0.0006167436849392187,
 -0.0023163561189554365,
 -0.017135645669543974,
 0.0003591065459019516,
 0.007178959997255522,
 -0.015740440524799164,
 0.004705643648762286,
 -0.02830996399137798,
 0.0033072682817259494,
 -0.00565691895249292,
 -0.005672773789241136,
 0.0025874697295302906,
 0.007274087620760849,
 -0.021131003062799813,
 -0.00943665466379266,
 0.0029790783298785603,
 0.021562248289357298,
 0.005060786310854187,
 0.019710430551786912,
 -0.00022672069050186825,
 -0.00

In [69]:
PINECONE_API_KEY= os.environ.get('PINECONE_API_KEY','')
PINECONE_API_ENV= os.environ.get('PINECONE_API_ENV','gcp-starter')

In [71]:
import pinecone

c:\Users\HP\genaivectordb\myenv\lib\site-packages\pinecone\index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [72]:
pinecone.init(api_key=PINECONE_API_KEY,environment=PINECONE_API_ENV)

In [73]:
index_name="test"

In [74]:
index = pinecone.Index('test')

In [76]:
#Embedding of all content
docsearh= Pinecone.from_texts([t.page_content for t in text_chunk],embedding,index_name='test')

In [77]:
docsearh

In [89]:
query = "The Transformer - model architecture."

In [85]:
docs= docsearh.similarity_search(query)

<bound method VectorStore.as_retriever of <langchain_community.vectorstores.pinecone.Pinecone object at 0x000001A1F6CDEDC0>>

In [82]:
docs

[Document(page_content='Figure 1: The Transformer - model architecture.\nwise fully connected feed-forward network. We employ a residual connection [ 10] around each of\nthe two sub-layers, followed by layer normalization [ 1]. That is, the output of each sub-layer is\nLayerNorm( x+ Sublayer( x)), where Sublayer(x)is the function implemented by the sub-layer\nitself. To facilitate these residual connections, all sub-layers in the model, as well as the embedding\nlayers, produce outputs of dimension dmodel = 512 .'),
 Document(page_content='architectures from the literature. We estimate the number of ﬂoating point operations used to train a\nmodel by multiplying the training time, the number of GPUs used, and an estimate of the sustained\nsingle-precision ﬂoating-point capacity of each GPU5.\n6.2 Model Variations\nTo evaluate the importance of different components of the Transformer, we varied our base model\nin different ways, measuring the change in performance on English-to-German tr

In [83]:
llm=OpenAI()

In [87]:
qa= RetrievalQA.from_chain_type(llm=llm,chain_type="stuff",retriever=docsearh.as_retriever())


In [90]:
qa.run(query)

' The Transformer is a model architecture that is based solely on attention mechanisms, which allows for more parallelization and has been shown to be superior in quality compared to models that use recurrence or convolution. It employs a residual connection and layer normalization, and all sub-layers and embedding layers produce outputs of dimension 512.'

In [97]:
import sys

while True:
    user_input=input("Input Prompt")
    if(user_input == "Exit"):
        print("Exiting")
        sys.exit()
    if user_input == '':
        continue
    result = qa({
        'query':user_input
    })
    print(f"Answer : {result['result']}")


Exiting


SystemExit: 

c:\Users\HP\genaivectordb\myenv\lib\site-packages\IPython\core\interactiveshell.py:3513: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
